In [41]:
#System and standard inputs
import sys
import importlib
import timeit
import time
import collections

#Adding correct paths to custom modules
sys.path.append('/Users/sasrey/AlphaTrade/gym_exchange')
sys.path.append('/Users/sasrey/AlphaTrade')
sys.path.append('/Users/sasrey/AlphaTrade/gymnax_exchange')

In [42]:
#Relevant Jax Imports
import jax.numpy as jnp
from jax import lax
import jax

In [656]:
#Direct Import of the Orderbook functions 
import jaxob.JaxOrderbook as job 
job=importlib.reload(job)
#OrderBook class import as written by Kang
import orderbook.orderbook
importlib.reload(orderbook.orderbook)
importlib.reload(orderbook.ordertree)
from orderbook.orderbook import OrderBook as cpuOrderBook
#Import (and reload for debugging) of the OrderBook class with jax backend (SF) should behave in the same way.
import jorderbook
importlib.reload(jorderbook)
from jorderbook import OrderBook as JaxOrderBook

#Import Data Mgt and Encoding/Decoding tools
#(The aim of these is to adjust for the case where no message is received but data ends up in the L2 data)
import gym_exchange.data_orderbook_adapter.data_pipeline as dpl
from gym_exchange.data_orderbook_adapter.decoder import Decoder
from gym_exchange.data_orderbook_adapter.encoder import Encoder

#Attempt at using the data pipeline tools and the encoder/decoder, doesn't really work well. 

dplOBJ=dpl.DataPipeline()
decodeddata=Decoder(10,40,dplOBJ.historical_data,dplOBJ.data_loader)
#Decoder Functionality: Takes the message data (dataloader) and the orderbook data (historical data).
#Creates and maintains an orderbook as implemented in the CPU version.
#Creates and maintains a "DataAdjuster" object, which considers the historical data (orderbook) to be the "true" state
#Inits the orderbook with the first state in the orderbook data (hist) gives imaginary list of IDs to the orders created:
#each entry in the orderbook is considered to be from a single order. 
output=decodeddata.process()
#Process function for Decoder: goes through N (horizon) messages to ascertain whether there are adidtional messages to consider. 

enc=Encoder(decodeddata)
#enc()

In [657]:
import pandas as pd
df_msg = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_message_50.csv", header=None)
df_ob = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_orderbook_50.csv", header=None)
orderbookLevels=50
df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})

In [658]:
data=jnp.array(df_ob.iloc[0,:]).reshape(int(orderbookLevels*2),2) ##Careful - converts to int32 in jnp array, meaning the filler values get truncated.
newarr=jnp.zeros((int(orderbookLevels*2),8))
initOB=newarr.at[:,3].set(data[:,0]).at[:,2].set(data[:,1]).at[:,0].set(1).at[0:orderbookLevels*4:2,1].set(-1).at[1:orderbookLevels*4:2,1].set(1).at[:,4].set(0).at[:,5].set(job.INITID).at[:,6].set(34200).at[:,7].set(0).astype('int32')


df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})
df_orders['TradeID']=0
col_order=['Type','Side','Quantity','Price','TradeID','OrderID','Time']
df_orders=df_orders[col_order]
df_orders_simple=df_orders[(df_orders['Type']!=6)&(df_orders['Type']!=7)&(df_orders['Type']!=5)]


In [659]:
test=df_orders["Time"].astype('string').str.split('.',expand=True)
df_orders[["TimeWhole","TimeDec"]]=test.astype('int32')#.rename(['TimeWhole','TimeDec'])
df_orders=df_orders.drop("Time",axis=1)
ordersJNP=jnp.array(df_orders)
print(ordersJNP.shape)

(290737, 8)


In [660]:
horizon=7076
#horizon=87
orderbook_1=JaxOrderBook(price_levels=100,orderQueueLen=100)
trades_init=orderbook_1.process_orders_array(initOB)
t1=time.time()
trades_msgs=orderbook_1.process_orders_array(ordersJNP[0:horizon]).block_until_ready()
t2=time.time()
#trades_msgs=orderbook_1.process_orders_array(ordersJNP[horizon-1:horizon])
print(t2-t1)

3.91018009185791


In [661]:
MINPRICE=-1.410065407e9
myOB=orderbook_1.get_L2_state()
L2OB=jnp.array(df_ob.iloc[horizon-1])
L2OB=L2OB.at[L2OB==MINPRICE].set(-1)


'''
print(ordersJNP[horizon])
print('##################   ORDERBOOK DELTA   ##########################')
print((myOB.reshape(100,4)[0:50,:]-L2OB.reshape(50,4)))#L2OB#-myOB
print('############################################')
print(L2OB.reshape(50,4))
print('############################################')
print(myOB.reshape(100,4)[0:60,:])
'''




"\nprint(ordersJNP[horizon])\nprint('##################   ORDERBOOK DELTA   ##########################')\nprint((myOB.reshape(100,4)[0:50,:]-L2OB.reshape(50,4)))#L2OB#-myOB\nprint('############################################')\nprint(L2OB.reshape(50,4))\nprint('############################################')\nprint(myOB.reshape(100,4)[0:60,:])\n"

## Load the original orderbook implementation on the CPU

In [662]:
def type_side_converter(type : int, side : int):
    side_ret = 'bid' if side ==1 else 'ask'
    if type==1:
        return 'limit',side_ret
    elif type==2:
        return 'cancel',side_ret
    elif type==3:
        return 'delete',side_ret
    elif type==4:
        side_ret = 'ask' if side ==1 else 'bid'
        return 'limit',side_ret
    else:
        return 'skip',side_ret

In [663]:
orderbook_2=cpuOrderBook()
limit_orders = []
order_id_list = [orderbook.orderbook.INITID + i for i in range(100)]
for i in range(50):
    trade_id = 10086
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'ask', 
          'quantity' : df_ob.iloc[0,4* i + 1], 
          'price' : df_ob.iloc[0,4*i],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
for i in range(50):
    trade_id = 10087
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'bid', 
          'quantity' : df_ob.iloc[0,4* i + 3], 
          'price' : df_ob.iloc[0,4*i+2],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
# Add init orders to order book
for order in limit_orders:
    # breakpoint()
    trades, order_id = orderbook_2.process_order(order, True, False)  

#print(orderbook_2)

In [664]:
##Taken largely from the Example.py file in the orderbook repository for the cpu
t1=time.time()
for index in range(horizon):
    l1 = df_msg.iloc[index,:]
    ttype,side=type_side_converter(l1[1],l1[5])
    quantity = l1[3]
    price = l1[4]
    trade_id = l1[2] # not sure, in the data it is order id
    order_id = trade_id
    timestamp = str(l1[0])
    message = {'type': ttype,'side': side,'quantity': quantity,'price': price,'trade_id': trade_id, "timestamp":timestamp, 'order_id':order_id}
    orderbook_2.processOrder(message,True,False)

t2=time.time()
        
print("The Final order is:")
print(message)

The Final order is:
{'type': 'limit', 'side': 'bid', 'quantity': 9, 'price': 2191600, 'trade_id': 26215850, 'timestamp': '35531.893087829', 'order_id': 26215850}


In [665]:
ordersJNP[horizon-1]

Array([        4,        -1,         9,   2191600,         0,  26215850,
           35531, 893087829], dtype=int32)

In [666]:
print(orderbook_1.orderbook_array[1])
print(orderbook_2.bids.get_price_list(2191600))

[[[       41   2191600         0  26215850     35531 891792458]
  [        9   2191600         0  26215850     35531 893087829]
  [       -1        -1        -1        -1        -1        -1]
  ...
  [       -1        -1        -1        -1        -1        -1]
  [       -1        -1        -1        -1        -1        -1]
  [       -1        -1        -1        -1        -1        -1]]

 [[       50   2191000         0  26162062     35525 361478742]
  [       -1        -1        -1        -1        -1        -1]
  [       -1        -1        -1        -1        -1        -1]
  ...
  [       -1        -1        -1        -1        -1        -1]
  [       -1        -1        -1        -1        -1        -1]
  [       -1        -1        -1        -1        -1        -1]]

 [[       50   2190700         0   7571084     34200 765595634]
  [       -1        -1        -1        -1        -1        -1]
  [       -1        -1        -1        -1        -1        -1]
  ...
  [       -1      

In [667]:
t2-t1

0.3839552402496338

In [668]:
from itertools import zip_longest
cpuOB=jnp.array(list(zip_longest(*orderbook_2.get_L2_state(), fillvalue=-1)))

print('#'*15+" Difference between the JAX (GPU) and Python (CPU) orderbook implementations"+'#'*15)
print('#'*15+" Horizon: "+str(horizon)+' Param2: '+'#'*15)
print((cpuOB[:50,:]-myOB.reshape(100,4)[0:50,:]))
print('#'*15+" Python (CPU) orderbook "+'#'*15)
print('#'*15+" Horizon: "+str(horizon)+' Param2: '+'#'*15)
print(cpuOB[:50,:])
print('#'*15+" JAX (GPU) orderbook "+'#'*15)
print('#'*15+" Horizon: "+str(horizon)+' Param2: '+'#'*15)
print(myOB.reshape(100,4)[0:50,:])

############### Difference between the JAX (GPU) and Python (CPU) orderbook implementations###############
############### Horizon: 7076 Param2: ###############
[[  0   0   0 -41]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [ 

In [669]:
testorder = {'type' : 'delete', 
            'side' : 'bid', 
          'quantity' : 300, 
          'price' : 2226400,
          'trade_id' : 0,
          'order_id' : 11,#orderbook.orderbook.INITID,
          "timestamp": '34200.1'}
orderbook_2.processOrder(testorder,from_data=True,verbose=False)
print(orderbook_2)

====================================== Asks ======================================
Quantity    30  |  Price  2900000  |  Order_ID   15000049  |  Time 34200.000000001
Quantity    16  |  Price  2887300  |  Order_ID   15000048  |  Time 34200.000000001
Quantity    15  |  Price  2850000  |  Order_ID   15000047  |  Time 34200.000000001
Quantity    10  |  Price  2796200  |  Order_ID   15000046  |  Time 34200.000000001
Quantity    50  |  Price  2780000  |  Order_ID   15000045  |  Time 34200.000000001
Quantity    20  |  Price  2750000  |  Order_ID   15000044  |  Time 34200.000000001
Quantity   100  |  Price  2675100  |  Order_ID   15000043  |  Time 34200.000000001
Quantity    10  |  Price  2675000  |  Order_ID   15000042  |  Time 34200.000000001
Quantity   100  |  Price  2674900  |  Order_ID   15000041  |  Time 34200.000000001
Quantity   100  |  Price  2674600  |  Order_ID   15000040  |  Time 34200.000000001
Quantity    76  |  Price  2656200  |  Order_ID   15000039  |  Time 34200.000000001
Quan